In [1]:
import json 
import pandas as pd 
import numpy as np 
import os 
import re 
import seaborn as sns 
import matplotlib.pyplot as plt
from omegaconf import OmegaConf 
from glob import glob 
os.chdir('../')

def load_df(log_dir):
    with open(log_dir, 'r') as f: 
        data = f.readlines()
    df = pd.DataFrame(list(pd.Series(data).map(eval).values))    
    return df 

def load_log_data(class_name, version, result_dir,method):

    df = pd.DataFrame()
    for i,v in enumerate(version):
        log_dir = os.path.join(result_dir,class_name,method,v,'log.txt')
        
        temp_df = load_df(log_dir)
        temp_df['hue'] = v 
        
        df = pd.concat([df,temp_df])
    
    return df 

def plot_log(data,class_name, metrics:list, figsize:tuple = (10,7)):
    
    fig, axes = plt.subplots(len(metrics),1,figsize=figsize)
    
    for ax, metric in zip(axes, metrics):
        sns.lineplot(
            x = 'epoch',
            y = metric,
            data = data,
            hue = data['hue'],
            ax = ax 
        )
      
    fig.suptitle(class_name)
    plt.show()
    
def read_txt(log_dir):
    with open(log_dir, 'r') as f: 
        data = f.readlines()    
    df = pd.DataFrame([eval(d) for d in data]) 
    return df 
pd.options.display.float_format = '{: .3f}'.format 

def temperature_matching(x):
    value = x.split('_')[-1]
    
    if value in ['BASELINE','focalloss']:
        return 0.05 
    elif value in ['T1','T2']:
        return float(value[-1])
    else:
        if len(value) ==1:
            return value 
        else:
            return re.search(r'T_(\d+\.\d+)', x).group(1)
    

# 전체 결과 

In [2]:
version = ['*-anomaly_ratio_0.0']

result_dir = './results'
dataset = ''
method = '*'
log_dirs = pd.Series(glob('/Volume/ProxyCore/results/*/*/*/*-anomaly_ratio_0.0/seed_0/result.txt'))
log_dirs = log_dirs[log_dirs.apply(lambda x : 'BASELINE' in x )].values
df = pd.DataFrame()

for log_dir in log_dirs:
    auroc_list = {}
    method = log_dir.split('/')[-6]
    dataset = log_dir.split('/')[-5]
    class_name = log_dir.split('/')[-4]
    exp_name = log_dir.split('/')[-3].split('-a')[0]
    

    
    data = load_df(log_dir)

    met = [d['pix_level']['average_precision'] for d in data['test_metrics']]    
        
    idx = np.argmax(met)
    
    result = pd.DataFrame(data['test_metrics'][idx])
    result = result.reset_index().melt(['index'])
    result.columns = ['metric','level','value']
    
    if exp_name == 'baseline_yen':
        method = 'softpatch-yen'
    anomaly_ratio = float(log_dir.split('/')[-3].split('_')[-1])
    
    for c in ['method','dataset','class_name','exp_name','anomaly_ratio']:
        result[c] = eval(c) 
    
    # Coreinit Option 
    if method == 'CoreInit':
        # Sampling Ratio 
        configs = OmegaConf.load(log_dir.replace('result.txt','configs.yaml'))
        sampling_ratio = configs.MODEL.params.sampling_ratio
        result['sampling_ratio'] = sampling_ratio 
        
        if 'pslabel_sampling_ratio' in configs.MODEL.params:
            pslabel_sampling_ratio = configs.MODEL.params.pslabel_sampling_ratio
            result['pslabel_sampling_ratio'] = pslabel_sampling_ratio
        else:
            result['pslabel_sampling_ratio'] = sampling_ratio
            
        if 'temperature' in configs.MODEL.params:
            result['Temperature'] = configs.MODEL.params.temperature
        else:
            result['Temperature'] = 0.05 
        
    df = pd.concat([df,result])

df = df[df['metric'].apply(lambda x : x in ['auroc','loco_auroc','average_precision'])].reset_index(drop=True)
df.loc[df['metric']=='loco_auroc','structural_anomalies'] = df[df['metric']=='loco_auroc']['value'].apply(lambda x : x['structural_anomalies'])
df.loc[df['metric']=='loco_auroc','logical_anomalies'] = df[df['metric']=='loco_auroc']['value'].apply(lambda x : x['logical_anomalies'])

# df = df.melt(['index','class','anomaly_ratio','exp_name'])
# # exp_d = {'Proxy_single_base' : 'ProxyNCA', 'Proxy_single_anchor_loss':'Proxy anchor', 'Proxy_nsoftmax' : 'Proxy nSoftmax'}
# # df['exp_name'] = df['exp_name'].map(exp_d)
# ind_d = {'img_level':'Image Level', 'pix_level' : 'Pixel Level'}
# df['index'] = df['index'].map(ind_d)

exp = df[
    (df['exp_name'].apply(lambda x : 'BASELINE' in x ))&
    (df['exp_name'].apply(lambda x : x not in ['BASELINE_CosineAnnealingLR', 'BASELINE_LR', 'BASELINE_Temperature_more', 'BASELINE_Temperature'] ))&
    (df['method']=='CoreInit')
    ].reset_index(drop=True)

exp['FocalLoss']    = exp['exp_name'].apply(lambda x : 'focalloss' if 'focalloss' in x else 'CrossEntropy')
exp['Augmentation'] = exp['exp_name'].apply(lambda x : 'None' if 'no_augment' in x else 'Augment')
exp['Label_core']   = exp['exp_name'].apply(lambda x : 'norm_core' if 'norm_core' in x else 'Core')
exp['Proxy_init']   = exp['exp_name'].apply(lambda x : 'rand_init' if 'rand_init' in x else 'coreset_init')

exp['method'] = 'CoreInit'

In [6]:
dataset = 'MVTecAD'
metric = 'average_precision'
level = 'pix_level'

index = ['class_name']
columns = ['method']
exp1= exp[
     (exp['dataset']==dataset) &
     (exp['Label_core']=='norm_core') & # norm_core, Core 
     (exp['FocalLoss']=='focalloss') & #CrossEntropy, focalloss 
     (exp['Proxy_init']=='coreset_init') & #rand_init coreset_init
     (exp['Augmentation']=='Augment') &
     (exp['Temperature']==0.1) &
     (exp['metric']==metric) &
     (exp['level']==level)
     ].pivot_table(index=index,columns=columns,values='value',aggfunc='mean')

base = df[
    (df['exp_name'].apply(lambda x : 'BASELINE' in x ))&
    (df['metric'].apply(lambda x : 'loco' not in x ))&
    (df['method'] != 'CoreInit')
    ].reset_index(drop=True).pivot_table(index=['class_name','level','dataset','metric','method'],columns=['exp_name'],values='value',aggfunc='max')
base = base.reset_index().melt(id_vars=['class_name','level','dataset','metric','method'])

base1= base[
    (base['dataset']==dataset) &
    (base['metric']==metric) &
    (base['level']== level)
].pivot_table(index=index,columns=columns,values='value')

result = pd.concat([base1,exp1],axis=1)
columns = ['FastFlow','ReverseDistillation','SPADE','PaDiM','PatchCore','CoreInit']
#result.loc['Average'] = result.mean()

In [10]:
result['CoreInit'] - result['PatchCore']

class_name
bottle        0.023
cable         0.030
capsule       0.016
carpet        0.115
grid          0.002
hazelnut      0.021
leather       0.056
metal_nut     0.001
pill         -0.025
screw        -0.004
tile          0.042
toothbrush    0.027
transistor   -0.021
wood         -0.012
zipper        0.016
dtype: float64

In [3]:
dataset = 'MVTecAD'
metric = 'average_precision'
level = 'pix_level'

index = ['pslabel_sampling_ratio']
columns = ['sampling_ratio']
exp1 = exp[
     (exp['dataset']==dataset) &
     (exp['Label_core']=='norm_core') & # norm_core, Core 
     (exp['FocalLoss']=='focalloss') & #CrossEntropy, focalloss 
     (exp['Proxy_init']=='coreset_init') & #rand_init coreset_init
     (exp['Augmentation']=='Augment') &
     (exp['Temperature']==0.05) &
     (exp['metric']==metric) &
     (exp['level']==level)
     ].pivot_table(index=index,columns=columns,values='value',aggfunc='mean')

In [4]:
exp1

sampling_ratio,0.010,0.100,0.250
pslabel_sampling_ratio,,,
0.010,0.631,NaN,NaN
0.100,NaN,0.653,NaN
0.250,0.634,0.642,0.644
